In [ ]:
import pandas as pd

# Загрузка TSV файла
df = pd.read_csv("data/external/news_tg_csv/telegram_news.tsv", sep="\t")

# Сохранение в CSV
df.to_csv("data/external/news_tg_csv/telegram_news_2.сsv", sep=",", index=False)

In [ ]:
import csv

input_file = "../data/100000_with_emb.csv"
output_file = "../data/30000_with_emb.csv"

# Открываем входной файл
with open(input_file, "r", newline="", encoding="utf-8") as infile:
    reader = list(csv.reader(infile))  # Читаем все строки в список

# Берем заголовок и последние 10 000 строк
header = reader[0]  # Заголовок (первая строка)
last_5000_rows = reader[-30000:]  # Последние 10 000 строк

# Записываем в новый файл
with open(output_file, "w", newline="", encoding="utf-8") as outfile:
    writer = csv.writer(outfile)
    writer.writerow(header)  # Записываем заголовок
    writer.writerows(last_5000_rows)  # Записываем данные

print("Готово! Последние 10 000 строк сохранены в", output_file)

In [ ]:
import pandas as pd

# Пусть input.csv — исходный файл, а output.csv — результирующий файл
input_file = 'input.csv'
output_file = 'output.csv'

# Список названий, при наличии которых строка будет удалена
names_to_remove = ["Раньше всех. Ну почти.", "ТАСС", "Интерфакс"]

# Читаем CSV файл
df = pd.read_csv(input_file)

# Фильтруем строки, оставляя только те, у которых в столбце channel_name нет указанных названий
filtered_df = df[~df['channel_name'].isin(names_to_remove)]

# Сохраняем результат в новый CSV файл (без добавления индекса)
filtered_df.to_csv(output_file, index=False)

print(f"Файл '{output_file}' успешно создан!")

In [ ]:
import pandas as pd
df = pd.read_csv("../data/external/news_tg_csv/telegram_news.сsv")

In [ ]:
df['datetime'].dtype

In [ ]:
import pandas as pd
df = pd.read_csv("../data/raw/news_with_emb.csv")
len(df)

In [ ]:
import requests
import pandas as pd
from datetime import datetime, timedelta

# Устанавливаем даты
end_date = datetime.now()
start_date = end_date - timedelta(days=6*365)

# Формируем URL для API ЦБ РФ
url = f'https://cbr.ru/hd_base/KeyRate/?UniDbQuery.Posted=True&UniDbQuery.From={start_date.strftime("%d.%m.%Y")}&UniDbQuery.To={end_date.strftime("%d.%m.%Y")}'

# Получаем страницу и парсим таблицу
response = requests.get(url)
tables = pd.read_html(response.text)

# Выбираем нужную таблицу
key_rate_table = tables[0]

# Сохраняем данные в CSV
key_rate_table.to_csv('key_rate_cbr.csv', index=False)

print("Данные успешно загружены и сохранены в файл 'key_rate_cbr.csv'")

In [ ]:
import requests
import pandas as pd
from datetime import datetime, timedelta
import xml.etree.ElementTree as ET

def get_urals_prices_cbr():
    # Формируем даты для запроса
    end_date = datetime.now()
    start_date = end_date - timedelta(days=365*10)  # за последние 10 лет
    
    # Формируем URL запроса
    url = "http://www.cbr.ru/scripts/XML_dynamic.asp"
    params = {
        'date_req1': start_date.strftime('%d/%m/%Y'),
        'date_req2': end_date.strftime('%d/%m/%Y'),
        'VAL_NM_RQ': 'R01239'  # Код для нефти Urals
    }
    
    try:
        # Выполняем запрос
        response = requests.get(url, params=params)
        response.raise_for_status()
        
        # Парсим XML
        root = ET.fromstring(response.content)
        
        # Собираем данные
        data = []
        for record in root.findall('Record'):
            date = record.get('Date')
            value = record.find('Value').text
            nominal = record.find('Nominal').text
            
            data.append({
                'Date': datetime.strptime(date, '%d.%m.%Y'),
                'Price': float(value.replace(',', '.')),
                'Nominal': int(nominal)
            })
        
        # Создаем DataFrame
        df = pd.DataFrame(data)
        
        # Сортируем по дате
        df = df.sort_values('Date')
        
        # Сохраняем в CSV
        df.to_csv('../../../data/external/macro/urals_prices_cbr.csv', index=False)
        
        print(f"Данные успешно загружены и сохранены в 'urals_prices_cbr.csv'")
        print(f"Количество записей: {len(df)}")
        print("\nПервые 5 записей:")
        print(df.head())
        
        return df
        
    except requests.exceptions.RequestException as e:
        print(f"Ошибка при запросе к API ЦБ: {e}")
        return None

if __name__ == "__main__":
    get_urals_prices_cbr()

In [ ]:
import yfinance as yf
import pandas as pd
from datetime import datetime, timedelta

def download_brent_prices():
    # Получаем текущую дату
    end_date = datetime.now()
    # Дата 10 лет назад
    start_date = end_date - timedelta(days=365*10)
    
    # Загружаем данные для нефти Brent (тикер BZ=F)
    brent = yf.download('BZ=F', 
                        start=start_date.strftime('%Y-%m-%d'),
                        end=end_date.strftime('%Y-%m-%d'))
    
    # Сохраняем в CSV файл
    brent.to_csv('extra_features/brent_prices_10y.csv')
    
    print(f"Данные сохранены в файл 'brent_prices_10y.csv'")
    print(f"Количество записей: {len(brent)}")
    
    # Показываем первые несколько строк
    print("\nПервые 5 записей:")
    print(brent.head())

if __name__ == "__main__":
    download_brent_prices()

In [ ]:
import requests
import pandas as pd
from datetime import datetime, timedelta
# Функция для загрузки данных о курсе доллара за заданный диапазон дат
def get_usd_to_rub_exchange_rate(start_date, end_date):
    base_url = "https://www.cbr.ru/scripts/XML_dynamic.asp"

    # Форматирование дат для запроса
    start_date_str = start_date.strftime('%d/%m/%Y')
    end_date_str = end_date.strftime('%d/%m/%Y')

    # Параметры запроса
    params = {
        'date_req1': start_date_str,
        'date_req2': end_date_str,
        'VAL_NM_RQ': 'R01235'  # Код валюты для USD
    }

    # Запрос к API ЦБ РФ
    response = requests.get(base_url, params=params)

    if response.status_code != 200:
        raise Exception(f"Ошибка при запросе данных: {response.status_code}")

    # Парсинг ответа XML
    data = pd.read_xml(response.content, xpath="//Record")

    # Преобразование данных в DataFrame
    data['Date'] = pd.to_datetime(data['Date'], format='%d.%m.%Y')
    data['Value'] = data['Value'].str.replace(',', '.').astype(float)

    # Переименование столбцов для удобства
    data = data.rename(columns={
        'Date': 'Дата',
        'Value': 'Курс USD к RUB'
    })

    return data

# Получение данных за последние 10 лет
end_date = datetime.now()
start_date = end_date - timedelta(days=10*365)

# Загрузка данных
try:
    exchange_rate_data = get_usd_to_rub_exchange_rate(start_date, end_date)
    print("Данные успешно загружены!")
    
    # Сохранение данных в CSV
    output_file = "../../../data/external/macro/usd_to_rub.csv"
    exchange_rate_data.to_csv(output_file, index=False)
    print("Данные сохранены в файл 'usd_to_rub.csv'")
except Exception as e:
    print(f"Произошла ошибка: {e}")

In [ ]:
import pandas as pd
# Загрузка файла с эмбеддингами
emb_file = "../data/external/text/news_with_emb.csv"
df = pd.read_csv(emb_file)

# Поиск колонки с эмбеддингами (предполагаем, что это единственная колонка с JSON-строками)
emb_cols = df.columns[df.apply(lambda x: x.dtype == 'object' and x.str.contains(r'^\[.*\]$').any())]

if len(emb_cols) == 1:
    # Переименование колонки
    old_name = emb_cols[0]
    df = df.rename(columns={old_name: 'embedding'})
    
    # Сохранение обновленного файла
    df.to_csv(emb_file, index=False)
    print(f"Колонка '{old_name}' успешно переименована в 'embedding'")
else:
    print("Не удалось однозначно определить колонку с эмбеддингами")


In [ ]:
import requests
import pandas as pd

def get_moex_listed_stocks():
    """
    Fetches a list of actively traded stocks (shares) from the Moscow Exchange API (TQBR board).

    Returns:
        pandas.DataFrame: DataFrame containing the Ticker (SECID) and Short Name (SHORTNAME)
                          of the listed stocks, or None if an error occurs.
    """
    # MOEX ISS API endpoint for securities traded on the TQBR board (main stock board)
    url = "https://iss.moex.com/iss/engines/stock/markets/shares/boards/TQBR/securities.json"
    
    params = {
        "iss.meta": "off",          # Turn off metadata block
        "iss.only": "securities",   # Get only the securities data block
        "securities.columns": "SECID,SHORTNAME,SECNAME,ISIN,PREVADMITTEDQUOTE" # Specify desired columns
    }

    try:
        response = requests.get(url, params=params)
        response.raise_for_status()  # Raise an exception for bad status codes (4xx or 5xx)
        
        data = response.json()
        
        # Extract securities data
        securities_data = data['securities']['data']
        # Extract column names
        columns = data['securities']['columns']
        
        # Create DataFrame
        df = pd.DataFrame(securities_data, columns=columns)
        
        # Filter out securities that are not currently admitted to trading (optional, but good practice)
        # PREVADMITTEDQUOTE is often the last traded price, non-null for traded stocks
        # A more robust filter might be needed depending on exact MOEX API nuances
        df_traded = df[df['PREVADMITTEDQUOTE'].notna()].copy()

        # Select relevant columns for the output
        df_result = df_traded[['SECID', 'SHORTNAME']].reset_index(drop=True)
        
        return df_result

    except requests.exceptions.RequestException as e:
        print(f"Ошибка запроса к MOEX API: {e}")
        return None
    except KeyError as e:
        print(f"Ошибка: Неожиданная структура ответа от MOEX API. Отсутствует ключ: {e}")
        return None
    except Exception as e:
        print(f"Произошла непредвиденная ошибка: {e}")
        return None

# --- Execution ---
print("Запрос списка акций с Московской Биржи (board TQBR)...")
stocks_df = get_moex_listed_stocks()

if stocks_df is not None:
    print(f"\nНайдено {len(stocks_df)} торгуемых акций на TQBR:")
    # Display the first few and last few rows for brevity
    with pd.option_context('display.max_rows', 10):
        print(stocks_df)
else:
    print("Не удалось получить список акций.")

